<a href="https://colab.research.google.com/github/strawndri/ic-drug-repurposing/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd

In [21]:
ppi = pd.read_csv("https://raw.githubusercontent.com/strawndri/ic-drug-repurposing/refs/heads/main/data/PPI.csv")
ppi.head()

,from,to,Source
0,A1BG,ABCC6,HINT|PINA|APID|BioGrid|Instruct
1,A1BG,ANXA7,HINT|HIPPIE|PINA|APID|BioGrid|Instruct|MINT
2,A1BG,CDKN1A,Signalink|HINT|PINA|APID|BioGrid|Instruct|MINT
3,A1BG,CRISP3,PINA|InWeb
4,A1BG,FDXR,BioPlex


In [20]:
ppi_human_virus = pd.read_csv('https://raw.githubusercontent.com/strawndri/ic-drug-repurposing/refs/heads/main/data/HVIDB_PPIs.csv')
ppi_human_virus.head()

,Uniprot_human,Uniprot_virus,EntryName_human,EntryName_virus,Organism_human,Organism_virus,Organism_Interactor_human,Organism_Interactor_virus,Experimental_System,Pubmed_ID,...,Short,HDF,HRF,Viral_family,Human_GeneName,Human_ProteinName,Human_GeneID,Virus_GeneName,Virus_ProteinName,Virus_GeneID
0,P62277,Q1HVI8,RS13_HUMAN,EBNA5_EBVA8,Homo sapiens (Human),Epstein-Barr virus (strain AG876) (HHV-4) (Hum...,9606,82830,two hybrid,22761553,...,HHV-4,no,no,Herpesviridae,RPS13,40S ribosomal protein S13,6207,EBNA-LP;EBNA5,Epstein-Barr nuclear antigen leader protein,5176191
1,Q8NFH4,P05954,NUP37_HUMAN,VPR_HV1RH,Homo sapiens (Human),Human immunodeficiency virus type 1 group M su...,9606,11701,Reactome Curated,21067998,...,HIV-1,no,no,Retroviridae,NUP37,Nucleoporin Nup37,79023,vpr,Protein Vpr,NaN
2,Q9UI26,Q2HRD4,IPO11_HUMAN,ORF4_HHV8P,Homo sapiens (Human),Human herpesvirus 8 type P (isolate GK18) (HHV...,9606,868565,affinity chromatography technology,25544563,...,HHV-8P,no,no,Herpesviridae,IPO11;RANBP11,Importin-11,51194,ORF4,Complement control protein,4961488
3,Q8TAE8,I6TAH8,G45IP_HUMAN,I6TAH8_I68A0,Homo sapiens (Human),Influenza A virus (strain A/Aichi/2/1968 H3N2),9606,387139,anti tag coimmunoprecipitation,28169297,...,H3N2,no,no,Orthomyxoviridae,GADD45GIP1;MRPL59;PLINP1;PRG6,Growth arrest and DNA damage-inducible protein...,90480,NP,Nucleoprotein,NaN
4,Q7Z3B4,P35962,NUP54_HUMAN,GAG_HV1Y2,Homo sapiens (Human),Human immunodeficiency virus type 1 group M su...,9606,362651,Reactome Curated,21067998,...,HIV-1,no,no,Retroviridae,NUP54,Nucleoporin p54,53371,gag,Gag polyprotein,NaN


In [60]:
selecao = (ppi_human_virus['Short'] == 'HIV-1') & (ppi_human_virus['Organism_virus'].str.contains('group M', na=False))
ppi_hiv1M = ppi_human_virus[selecao]

In [61]:
ppi_hiv1M.reset_index(inplace=True)
ppi_hiv1M = ppi_hiv1M[['Uniprot_human', 'Uniprot_virus', 'Human_GeneName']]

ppi_hiv1M

,Uniprot_human,Uniprot_virus,Human_GeneName
0,Q8NFH4,P05954,NUP37
1,Q7Z3B4,P35962,NUP54
2,Q10567,P04604,AP1B1;ADTB1;BAM22;CLAPB2
3,O95793-2,P04591-PRO_0000038594,STAU1;STAU
4,Q15046,P04591,KARS1;KARS;KIAA0070
...,...,...,...
10753,P01903,P04585,HLA-DRA;HLA-DRA1
10754,Q16630,P12493,CPSF6;CFIM68
10755,P04439,P04583,HLA-A;HLAA
10756,P01911,P04585,HLA-DRB1
